In [27]:
from nltk import word_tokenize
from nltk import wordnet as wn
import re
import pandas as pd
import os
import nltk

In [28]:
os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Projects/Web Scraping Project - EEOC Articles')
Url = r'EEOC Article Study - Industry Values.xlsx'
df1 = pd.read_excel(Url)
df2 = df1.set_index('Keys') # Set the Major Industry Group as the index. 
df3 = df2.iloc[0:22, :]     # Limit the scope of the dataframe.  Function is picking up nan values which is causing
                            # an error. 

In [29]:
# Dictionary of Industry Keys and Values

In [30]:
def Create_Industry_Dictionary():
    os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Projects/Web Scraping Project - EEOC Articles')
    Url = r'EEOC Article Study - Industry Values.xlsx'
    df1 = pd.read_excel(Url)
    df2 = df1.set_index('Keys')  
    dataframe = df2.iloc[0:22, :]      
                             
    Dict_major = {}
    
    # for each major industry value (column A) in the dataframe
    for major in dataframe.index:
        
        # Define the minor industry as the row values. 
        df_minor = dataframe.loc[major]
        
        # Define the industry value lvl 2 and 3 to include in the keys of your dictionary (for descriptive purp)
        Industry_lvl_2 = df_minor['Division']
        Industry_lvl_3 = df_minor['Column 1']

        # Define the list to catch the synsets generated by the below function. 
        Synset_list = []
        
        # For the value in the dataframe, the key being the value in column B 'Division'
        for value in df_minor[1:]:
            # Verify that it is an instance of a string as we have None values in the dataframe. 
            if isinstance(value, str):
                # Convert the value to lowercase. 
                value_lower = value.lower()
                # Generate synsets for this value.
                Synset = wn.wordnet.synsets(value_lower)
                # Extract the word from the synset object ('word.n.01')
                Lemma_names = [x.lemma_names() for x in Synset]
                # Lemman_names is a list of lists.  Iterate over each list. 
                for List in Lemma_names:
                    # Get the words in each sub list. 
                    for word in List:
                        # We want to end up with a Set of unique values.  Therefore, check to see 
                        # if the word is already in our list. 
                        if word not in Synset_list:
                            # If not, then append the word to our list. 
                            Synset_list.append(word)
        
        # Create the name of the Major Industry Group that will constitute the Keys of our Dict. 
        Industry_identifier = (str(major) + ' ' + Industry_lvl_2 + ' - ' + Industry_lvl_3)
        # Join the Keys with our matching values. 
        Dict_major[Industry_identifier] = Synset_list
    
    # Return our completed Industry Dictionary
    return Dict_major

In [31]:
Industry_Dictionary = Create_Industry_Dictionary(df3)

In [32]:
Industry_Dictionary  # BOOM!

{'Major Group 01-09 Agriculture - Agriculture': ['agribusiness',
  'agriculture',
  'factory_farm',
  'farming',
  'husbandry',
  'Department_of_Agriculture',
  'Agriculture_Department',
  'Agriculture',
  'USDA',
  'farm',
  'grow',
  'raise',
  'produce',
  'land',
  'agrarian',
  'agricultural',
  'hunt',
  'hunting',
  'search',
  'run',
  'hunt_down',
  'track_down',
  'hound',
  'trace',
  'fishing',
  'sportfishing',
  'fish',
  'angle'],
 'Major Group 10-14 Mining - Mining': ['mining',
  'excavation',
  'minelaying',
  'mine',
  'miner',
  'mineworker',
  'coal',
  'ember',
  'char',
  'copper',
  'Cu',
  'atomic_number_29',
  'bull',
  'cop',
  'fuzz',
  'pig',
  'copper_color',
  'zinc',
  'Zn',
  'atomic_number_30',
  'gold',
  'amber',
  'Au',
  'atomic_number_79',
  'golden',
  'gilded',
  'aureate',
  'gilt'],
 'Major Group 15-17 Construction - Construction': ['construction',
  'building',
  'grammatical_construction',
  'expression',
  'mental_synthesis',
  'structure',


In [33]:
# Count of Matching Values From Text

In [34]:
def Get_Text_Industry_Count(Token_list, Dict_Industries):

    Word_match_count = {}
    
    for token in Token_list:  

        for major_group in Dict_Industries:                         

            if token in Dict_Industries[major_group]:
            
            #else: do nothing
                
                if major_group in Word_match_count.keys():
                
                    if token in Word_match_count[major_group]:
                        Word_match_count[major_group] = Word_match_count[major_group] + [token]
                    
                    else: #if the major group is there, but not toke, then add token. 
                        Word_match_count[major_group] = [token]
                
                else: #if the major group is not there, then we need to create it and add the token.  
                    Word_match_count[major_group] = [token]                    
                    
    
    # For each key, value pair, take the length of the values for a given key as the # of matches for that key. 
    for x in Word_match_count:
        Word_match_count[x] = len(Word_match_count[x])

    # For Major Group Legal Services, take half the count to compensate for the fact that we are 
    # only reviewing legal texts biasing the matches for any given industry.
    
        if '81' in x:
            Word_match_count[x] = Word_match_count[x] * 0.5           
                                             
    # Create a Dataframe of our Dictionary Values & Sort to rank the industry groups by num matches
    # Keep the ranking separate from the prediction so that you can investigate the results. 
    
    df = pd.DataFrame(Word_match_count, index = [1])
    df_tran = pd.DataFrame.transpose(df)
    df_sorted = df_tran.sort_values(1, ascending = False)
    
    return df_sorted


In [35]:
# Get Industry Prediction

In [36]:
def Industry_prediction(Obj_from_industry_classifier):
    # Return the top row as the Prediction from our Industry Classifier function. 
    Top_row = Obj_from_industry_classifier.iloc[0]
    Industry = Top_row.name
    return Industry

In [37]:
# Import Text Cleaning Packages

In [38]:
chdir = os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Modules/')
import ccirelli2_text_analysis_module as cc

In [39]:
# Import Texts

In [40]:
chdir = os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Projects/Web Scraping project - EEOC Articles/EEOC Articles/')
Cdw = os.getcwd()
File_name_list = os.listdir()

def get_Dir_list(File):
        List = []
        for x in File:    
            if 'txt' in x:                                    # Confirm a txt file
                y = Cdw + '\\' + x  
                List.append(y)
        return List

Dir_list = get_Dir_list(File_name_list)

In [41]:
# Put It All Together

In [42]:
def predict_industry_pipeline(Text, Industry_dataframe):
    Text_tokenized = word_tokenize(Text)
    Clean_text = text_cleaning_pipeline(Text_tokenized)
    Industry_Dictionary = Create_Industry_Dictionary(Industry_dataframe)
    Industry_classifier = get_industry(Clean_text, Dict_Industries_dict)
    Industry_prediction = Industry_prediction(Industry_classifier)
    return Industry_prediction

In [43]:
# Loop over individual texts to get prediction

In [52]:
chdir = os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Projects/Web Scraping project - EEOC Articles/EEOC Articles/')
Url_name_list = os.listdir()

for x in Url_name_list[:10]:
    if '.txt' in x:
        File = open(x, 'rb')
        Text_bytes = File.read()
        Text_string = str(Text_bytes)
        
        Text_tokenized = nltk.word_tokenize(Text_string)
        Text_strip_punct = cc.strip_punctuation(Text_tokenized)
        Text_strip_stop_words = cc.strip_stop_words(Text_strip_punct)
        Text_strip_forwardslash = cc.strip_tokens_forwardSlash_x2(Text_strip_stop_words)
        Text_strip_two_variable_tokens = cc.strip_two_variable_tokens(Text_strip_forwardslash)
        Text_get_isalpha = cc.get_isalpha(Text_strip_two_variable_tokens)
        Final_text = [x.lower() for x in Text_get_isalpha]

        
        Industry_Dictionary = Create_Industry_Dictionary(df3)
        Industry_count = Get_Text_Industry_Count(Final_text, Industry_Dictionary)
        print(Industry_prediction(Industry_count))
        
        #### Something is wrong with the Get_Text_Industry_Count Function.  It is returning empty dataframes. 
        


Major Group 87 Services - Accounting
Major Group 20-39 Manufacturing - Manufacture
Major Group 81 Services - Law firm
Major Group 87 Services - Accounting
Major Group 91-99 Services - Government
Major Group 50-51 Wholesale - Wholesale
Major Group 91-99 Services - Government
Major Group 60-67 Finance - Bank 
Major Group 81 Services - Law firm
Major Group 73 Services - Business Services
